**Create requirement.txt file**

In [ ]:
# Write content to a text file
with open('requirements.txt', 'w') as f:
    f.write("""
    langchain
    streamlit
    openai
    huggingface_hub
    python-dotenv
    streamlit""") # using triple double inverted comma for mutiple entries

**Working on ipynb file**

In [ ]:
!pip install langchain
from langchain.llms import OpenAI

import os
os.environ['OPEN_AI_API_KEY']="sk-proj-ZlvuERRBPh05EY9f6NDYT3BlbkFJ8gQCYk2vAhkvYunitJgT"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
!pip install OpenAI
llm=OpenAI(openai_api_key=os.environ["OPEN_AI_API_KEY"],temperature=0.6) # the more value of the temperature towards 1 the more your model will be creative

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
text="What is the capital of india"
print(llm.predict(text))  # Not running because of money, Here we are working with open ai

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]="Here we have to mention token code" # It is asking for money

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64}) # when search for model on hugging face you will get it as a title
#max string length will be 64

In [ ]:
output=llm_huggingface.predict("Can you tell me the capital of Russia")
print(output) # here we are working with hugging face

**Prompt Templates and LLMChain**

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="India")

In [ ]:
from langchain.chains import LLMChain

In [ ]:
chain=LLMChain(llm=llm,prompt=prompt_template)
chain.run("India")

**Combining Multiple chains using simple Sequential chain**

In [ ]:
capital_prompt=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of {country}") # 1 template

capital_chain=LLMChain(llm=llm,prompt=capital_prompt)

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}") # 2 template

famous_chain=LLMChain(llm=llm,prompt=famous_template)

from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

**Sequential Chain**

In [ ]:
capital_prompt=PromptTemplate(input_variables=['country'],
template="Please tell me the capital of {country}") # 1 template

capital_chain=LLMChain(llm=llm,prompt=capital_prompt,output_key="capital")

famous_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some amazing places to visit in {capital}") # 2 template

famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
                    input_variables=['country'],
                    output_variables=['capital','places'])
chain.run("India")

**Chatmodels with ChatOpenAI**

In [ ]:
# this help us to create conversational chatbot.

from langchain.chat_models import ChatOpenAI

from langchain.schema import AIMessage,HumanMessage,SystemMessage
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_AI_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [ ]:
# let's create schema in a form of list
chatllm([
SystemMessage(content='You are a comedian AI assistant'),
HumanMessage(content='Tell me a joke'),
AIMessage(content='Why did the chicken cross the road?')
])

**Prompt Template + LLM + Output Parsers**

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class Commaseparatedoutput(BaseOutputParser):
  def parse(self, text: str):
      return text.strip().split(",")


template='You are a helpful assistant.When the user gives any input, you should generate 5 words synonyms in a comma separated list.'
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [ ]:
# | it is showing chained up
chain=chatprompt|chatllm|Commaseparatedoutput()
chain.invoke({"text":"intelligent"}) # now this will show me 5 synonyms of intelligent

**Create app.py file to deploy it on streamlit**

In [ ]:
from langchain.llms import OpenAI

from dotenv import load_dotenv
load_dotenv()

import streamlit as st
import os

# create function to load OpenAI model to get response

def get_openai_response(question):
    llm=OpenAI(openai_api_=os.getenv("OPEN API KEY"),key,model_name="text-davinci-003",temperature=0.5)
    llm_response=llm(question)
    return llm_response

# initialize our streamlit app

st.set_page_config(page_title="Q&A Demo")

st.header("Langchain Aplication")

# capture input text

input=st.text_input("Input: ",key="input")
response=get_openai_response(input)

# create submit button

submit=st.button("Ask the question")

# If ask button is clicked

if submit:
    st.subheader("The Response is")
    st.write(response)